In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn.functional as F

In [3]:
data = np.load('/Users/divyanshuchaudhari/Documents/myFaultNet/Dataset/CWRU files/signal_data.npy')
labels = np.load('/Users/divyanshuchaudhari/Documents/myFaultNet/Dataset/CWRU files/signal_data_labels.npy')

In [4]:
# x=data[:,0:1600]

In [5]:
# Generating white noise
# Calculating the SNR
white_noise_1 = np.clip(np.random.normal(0, 1, data.shape), -0.5, 0.5) # -4.5
signal_energy = np.mean(data**2, axis = 1)
noise_energy = np.mean(white_noise_1**2, axis = 1)
SNR =  10*np.log10(signal_energy/noise_energy)
print(np.mean(SNR), np.max(SNR), np.min(SNR))
###
white_noise_2 = np.clip(np.random.normal(0, 1, data.shape), -0.4, 0.4) # -2.9
signal_energy = np.mean(data**2, axis = 1)
noise_energy = np.mean(white_noise_2**2, axis = 1)
SNR =  10*np.log10(signal_energy/noise_energy)
print(np.mean(SNR), np.max(SNR), np.min(SNR))
###
white_noise_3 = np.clip(np.random.normal(0, 1, data.shape), -0.3, 0.3) # -0.6
signal_energy = np.mean(data**2, axis = 1)
noise_energy = np.mean(white_noise_3**2, axis = 1)
SNR =  10*np.log10(signal_energy/noise_energy)
print(np.mean(SNR), np.max(SNR), np.min(SNR))
###
white_noise_4 = np.clip(np.random.normal(0, 1, data.shape), -0.2, 0.2) # 2.6
signal_energy = np.mean(data**2, axis = 1)
noise_energy = np.mean(white_noise_4**2, axis = 1)
SNR =  10*np.log10(signal_energy/noise_energy)
print(np.mean(SNR), np.max(SNR), np.min(SNR))
###
white_noise_5 = np.clip(np.random.normal(0, 1, data.shape), -0.15, 0.15) # 5
signal_energy = np.mean(data**2, axis = 1)
noise_energy = np.mean(white_noise_5**2, axis = 1)
SNR =  10*np.log10(signal_energy/noise_energy)
print(np.mean(SNR), np.max(SNR), np.min(SNR))
###
white_noise_6 = np.clip(np.random.normal(0, 1, data.shape), -0.1, 0.1) # 8.4
signal_energy = np.mean(data**2, axis = 1)
noise_energy = np.mean(white_noise_6**2, axis = 1)
SNR =  10*np.log10(signal_energy/noise_energy)
print(np.mean(SNR), np.max(SNR), np.min(SNR))
###
white_noise_7 = np.clip(np.random.normal(0, 1, data.shape), -0.075, 0.075) # 10.8
signal_energy = np.mean(data**2, axis = 1)
noise_energy = np.mean(white_noise_7**2, axis = 1)
SNR =  10*np.log10(signal_energy/noise_energy)
print(np.mean(SNR), np.max(SNR), np.min(SNR))

-4.528861845149553 8.908365860788146 -17.413081677628956
-2.8757894206644328 10.506660818683983 -15.75374890740285
-0.6499507294841451 12.809766599008338 -13.457179999464108
2.610832005922203 16.073202789348176 -10.139525035896227
4.982436120855565 18.399744575313374 -7.869631265110851
8.381616182634955 21.81086354493507 -4.472073588289229
10.819770537548125 24.253876238973294 -2.036373891718241


In [6]:
# Noisy data
data1 = data + white_noise_1
data2 = data + white_noise_2
data3 = data + white_noise_3
data4 = data + white_noise_4
data5 = data + white_noise_5
data6 = data + white_noise_6
data7 = data + white_noise_7

In [7]:
x = data1

In [8]:
def mean(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.mean(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def median(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.median(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def sig_image(data,size):
    X=np.zeros((data.shape[0],size,size))
    for i in range(data.shape[0]):
        X[i]=(data[i,:].reshape(size,size))
    return X.astype(np.float16)

In [9]:
channel_mean=(mean(x,10)).astype(np.float16)
x_m=sig_image(channel_mean,40)
channel_median=(median(x,10)).astype(np.float16)
x_md=sig_image(x,40)

In [10]:
x_n=sig_image(x,40)

In [11]:
x_n.shape

(2800, 40, 40)

In [12]:
x_m.shape

(2800, 40, 40)

In [13]:
X=np.stack((x_n,x_m,x_md),axis=1).astype(np.float16)

In [14]:
X.shape

(2800, 3, 40, 40)

In [15]:
from sklearn.model_selection import train_test_split
trainx, testx, trainlabel, testlabel = train_test_split(X, labels, test_size=0.2, random_state=20)

In [16]:
sig_train, sig_test = trainx,testx
lab_train, lab_test = trainlabel,testlabel

In [17]:
sig_train = torch.from_numpy(sig_train)
sig_test = torch.from_numpy(sig_test)
lab_train= torch.from_numpy(lab_train)
lab_test = torch.from_numpy(lab_test)

In [18]:
import torch.utils.data as data_utils
batch_size = 128 
train_tensor = data_utils.TensorDataset(sig_train, lab_train) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [19]:
batch_size = 1024
test_tensor = data_utils.TensorDataset(sig_test, lab_test) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)

In [20]:
sig_train.size()

torch.Size([2240, 3, 40, 40])

In [21]:
sig_test.size()

torch.Size([560, 3, 40, 40])

In [22]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4,stride=1,padding = 1) #o/p: 32x39x39
        self.mp1 = nn.MaxPool2d(kernel_size=4,stride=2) #o/p:  32x17x17
        self.conv2 = nn.Conv2d(32,64, kernel_size=4,stride =1) #o/p: 64x14x14
        self.mp2 = nn.MaxPool2d(kernel_size=4,stride=2) #o/p: 64x6x6                                                                                                                                                                        
        self.fc1= nn.Linear(2304,256)  
        self.dp1 = nn.Dropout(p=0.25)
        self.fc2 = nn.Linear(256,10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp1(self.conv1(x)))
        x = F.relu(self.mp2(self.conv2(x)))
        x = x.view(in_size,-1)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [23]:
cnn = CNN().double()

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

In [25]:
num_epochs = 100

In [26]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # Run the forward pass
        #Here the size of label is [N,1] where N is batch-size
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        
        # loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        
        loss.backward() # Calculates the gradient and stores it
        optimizer.step() # Picks up the gradient=> performs optimization and updates the weights
        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list.append(correct / total)

        if (epoch+1) % 5 == 0 or epoch==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Train Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
        

Epoch [1/100], Step [1/18], Loss: 2.2985, Train Accuracy: 10.16%
Epoch [1/100], Step [2/18], Loss: 2.3839, Train Accuracy: 7.81%
Epoch [1/100], Step [3/18], Loss: 2.2476, Train Accuracy: 10.94%
Epoch [1/100], Step [4/18], Loss: 2.2239, Train Accuracy: 21.09%
Epoch [1/100], Step [5/18], Loss: 2.2238, Train Accuracy: 15.62%
Epoch [1/100], Step [6/18], Loss: 2.1730, Train Accuracy: 17.97%
Epoch [1/100], Step [7/18], Loss: 2.1488, Train Accuracy: 13.28%
Epoch [1/100], Step [8/18], Loss: 2.1417, Train Accuracy: 16.41%
Epoch [1/100], Step [9/18], Loss: 2.0593, Train Accuracy: 17.97%
Epoch [1/100], Step [10/18], Loss: 1.9246, Train Accuracy: 35.16%
Epoch [1/100], Step [11/18], Loss: 1.9782, Train Accuracy: 25.00%
Epoch [1/100], Step [12/18], Loss: 1.9429, Train Accuracy: 27.34%
Epoch [1/100], Step [13/18], Loss: 1.8339, Train Accuracy: 32.81%
Epoch [1/100], Step [14/18], Loss: 1.7031, Train Accuracy: 38.28%
Epoch [1/100], Step [15/18], Loss: 1.8723, Train Accuracy: 32.03%
Epoch [1/100], Step 

In [27]:
total_step = len(test_loader)
# print(total_step)
loss_list_test = []
acc_list_test = []
cnn.eval()
with torch.no_grad():
    for i, (signals, labels) in enumerate(test_loader):
        print(i)
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        loss_list_test.append(loss.item())
        # if epoch%10 ==0:
        #     print(loss)
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list_test.append(correct / total)
        if (epoch) % 1 == 0:
            print('Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(loss.item(),
                          (correct / total) * 100))

0
Loss: 0.8092, Accuracy: 74.29%
